# Materials associated with the paper: 

Cao, W., Williams, S., Flament, N., Zahirovic, S., Scotese, C., and Müller, R. D., 2018. Paleolatitudinal distribution of lithologic indicators of climate in a paleogeographic framework. Geological Magazine, 1-24. doi:10.1017/S0016756818000110.

### This Jupyter notebook is used to convert lithologic data from tabular foramt into shapefile format

The code in this notebook is written in Python 2.7. It utilises the standard scientific Python modules including numpy, pandas, glob, os and the open source modules pygplates which provides a Python API to the GPlates software (https://www.gplates.org/). Here, the source code has been modified to Python 3.7 by Behnam Sadeghi.


## Import Python modules

In [1]:
import pygplates
import pandas as pd
import numpy as np
import glob, os
os.environ['PROJ_LIB'] = r'C:\ProgramData\Anaconda3\pkgs\proj4-5.1.0-hfa6e2cd_1\Library\share'

## Read the input data and models

In [2]:
# read data
file_list = glob.glob('../Data/Original_Data/*.csv')

for filename in file_list[5:]:
    print (os.path.basename(filename)[:5])

age_range_list = [(541.0,509),  # Early Cambrian
           (509,485.4),   # Middle and Late Cambrian (509,485.4)
           (485.4,470.0),   # Early Ordovician (485.4,470.0)
           (470.0,443.8),   # Middle and Late Ordovician (470.0,443.8)
           (443.8,419.2),   # Silurian (443.8,419.2)
           (419,393),   # Early Devonian
           (393,388),   # Middle Devonian (Eifelian)
           (388,383),   # Middle Devonian (Givetian)
           (383,359),   # Late Devonian
           (359,331),   # Early Carboniferous (Tournaisian-Visean)
           (331,323),   # Early Carboniferous (Serpukhovian)
           (323,307),   # Late Carboniferous (Bashkirian-Moscovian)
           (307,299),   # Late Carboniferous (Kasimovian-Gzhelian)
           (299,290),   # Early Permian (Asselian-Sakmarian)
           (290,252),   # Middle-Late Permian (Artinskian-Lopingian)
           (252,247),   # Early Triassic
           (247,237),   # Middle Triassic
           (237,201),   # Late Triassic
           (201,164),   # Early and Middle Jurassic
           (164,145),   # Late Jurassic
           (145,113),   # Early Cretaceous (Berriasian-Aptian)
           (113,89.8),   # Late Cretaceous (Albian-Turonian)
           (89.8,66.0),   # Late Cretaceous (Coniacian-Maastrichtian)
           (66.0,56.0),   # Paleocene
           (56.0,47.8),  # Early Eocene (Ypresian?)
           (47.8,33.9),  # Middle and Late Eocene
           (33.9,23.0),  # Oligocene
           (23.0,5.3),    # Miocene
           (0.01,-0.01)]    

i = 0
for i in np.arange(5,29,1):
    recon_time = np.mean(age_range_list[i])
    print (recon_time)

Map06
Map07
Map08
Map09
Map10
Map11
Map12
Map13
Map14
Map15
Map16
Map17
Map18
Map19
Map20
Map21
Map22
Map23
Map24
Map25
Map26
Map27
Map28
Map29
406.0
390.5
385.5
371.0
345.0
327.0
315.0
303.0
294.5
271.0
249.5
242.0
219.0
182.5
154.5
129.0
101.4
77.9
61.0
51.9
40.849999999999994
28.45
14.15
0.0


In [3]:
# Metthews et al., 2016 static polygons
continental_polygons = pygplates.FeatureCollection('../Data/Tectonic_model/StaticPolys/PresentDay_StaticPlatePolygons_Matthews++.shp')
# Matthews et al., 2016 continental polygons
#continental_polygons = pygplates.FeatureCollection('/Users/wenchaocao/Research/6_CookieCutting_Corrections/Matthews++_410-0Ma_latest/ContPolys/PresentDay_ContinentalPolygons_Matthews++.shp')

# empty rotation file
input_rotation_filename = ['../Data/Tectonic_model/New_Rotation.rot']
rotation_model = pygplates.RotationModel(input_rotation_filename)

In [4]:
# define a function

def paleolith2shapefile(filename,age_range):
    
    df = pd.read_csv(filename,sep=',',encoding='latin1')
    point_features = []
    # put the points into a feature collection, using Lat,Long coordinates from dataframe
    for index,row in df.iterrows():
        point = pygplates.PointOnSphere(float(row.LAT),float(row.LONG))
        #print row.LONG
        point_feature = pygplates.Feature()
        point_feature.set_geometry(point)
        point_feature.set_shapefile_attribute('MapNum',os.path.basename(filename)[3:5])
        point_feature.set_shapefile_attribute('LithCode',row.LithologyCode)
        point_feature.set_shapefile_attribute('LithID',row.LithNumber)
        point_feature.set_shapefile_attribute('OldIDNumber',row.OldIDNumber)
        point_feature.set_shapefile_attribute('ModernLAT',row.LAT)
        point_feature.set_shapefile_attribute('NS',row.NS)
        point_feature.set_shapefile_attribute('ModernLONG',row.LONG)
        point_feature.set_shapefile_attribute('EW',row.EW)
        point_feature.set_shapefile_attribute('Continent',row.Continent)
        point_feature.set_shapefile_attribute('Country',row.Country)
        point_feature.set_shapefile_attribute('GeogComm',row.GeogComments)
        point_feature.set_shapefile_attribute('LMU',row.LMU)
        point_feature.set_shapefile_attribute('Period',row.Period)
        point_feature.set_shapefile_attribute('Stage',row.Stage)
        point_feature.set_shapefile_attribute('AgeComm',row.AgeComments)
        point_feature.set_shapefile_attribute('Lithology',row.Lithology)
        point_feature.set_shapefile_attribute('Formation',row.Formation)
        point_feature.set_shapefile_attribute('LithComm',row.LithComments)
        point_feature.set_shapefile_attribute('PrimRef',row.PrimaryReference)
        point_feature.set_shapefile_attribute('SeeAlso',row.SeeAlso)
        point_feature.set_valid_time(age_range[0],age_range[1]) #age_range[0],age_range[1] # age_range[1]+0.01
        point_features.append(point_feature)

    # The partition points function can then be used as before
    partitioned_point_features = []
    partitioned_point_features = pygplates.partition_into_plates(continental_polygons,
                                                           rotation_model,
                                                           point_features)
    
    # If partitioned feature has plate id = 0, we throw it away 
    partitioned_point_features_not_zero = []
    for point_feature in partitioned_point_features:
        if point_feature.get_reconstruction_plate_id() != 0:
            partitioned_point_features_not_zero.append(point_feature)
    
    return (partitioned_point_features_not_zero)

## Save all the data into one shapefile

In [5]:
features_collection = []

for filename,age_range in zip(file_list[5:],age_range_list[5:]): #zip(file_list[5:],age_range_list[5:])
    print (filename, age_range)
    
    partitioned_point_features_not_zero= []
    partitioned_point_features_not_zero = paleolith2shapefile(filename,age_range)
    
    print (len(partitioned_point_features_not_zero))
    
    point = []
    for point in partitioned_point_features_not_zero:
        features_collection.append(point)

output_feature_collection = pygplates.FeatureCollection(features_collection)
output_feature_collection.write('Output/ProcessingData_1/PresentDay_LithData_Matthews2016_410-0Ma_AllTypes.shp')

../Data/Original_Data\Map06 LDevon v5.csv (419, 393)
76
../Data/Original_Data\Map07 Eifel v5.csv (393, 388)
63
../Data/Original_Data\Map08 Givet v5.csv (388, 383)
73
../Data/Original_Data\Map09 UDevon v5.csv (383, 359)
138
../Data/Original_Data\Map10 TourVise v4.csv (359, 331)
458
../Data/Original_Data\Map11 Serpuk v4.csv (331, 323)
209
../Data/Original_Data\Map12 BashMosc v4.csv (323, 307)
570
../Data/Original_Data\Map13 Steph v4.csv (307, 299)
396
../Data/Original_Data\Map14  AssSak v4.csv (299, 290)
513
../Data/Original_Data\Map15 ArtinLoping v4.csv (290, 252)
599
../Data/Original_Data\Map16 LTrias v4.csv (252, 247)
95
../Data/Original_Data\Map17 MTrias v4.csv (247, 237)
121
../Data/Original_Data\Map18 UTrias v4.csv (237, 201)
430
../Data/Original_Data\Map19 LMJuras v4.csv (201, 164)
705
../Data/Original_Data\Map20 UJuras v4.csv (164, 145)
270
../Data/Original_Data\Map21 preAlbian  v6.csv (145, 113)
505
../Data/Original_Data\Map22 AlbianTuron v5.csv (113, 89.8)
440
../Data/Original_

## Save coals, evaporties and glacial deposits as one shpefile or save them individually

In [6]:
features_collection = []

for filename,age_range in zip(file_list[5:],age_range_list[5:]): #zip(file_list[5:],age_range_list[5:])
    print (filename, age_range)
    
    partitioned_point_features_not_zero= []
    partitioned_point_features_not_zero = paleolith2shapefile(filename,age_range)
        
    ################### save all types of data individually
    #output_feature_collection = pygplates.FeatureCollection(point_features)
    #output_feature_collection.write('PresentDay_LithData_Matthews2016_AllTypes_%sMa.shp' % np.mean(age_range))
    
    ################### save coals, evaporites and TDG
    point_features_onetype = []
    for point in partitioned_point_features_not_zero:
        if point.get_shapefile_attribute('LithCode')=='C':  # coals
        #if point.get_shapefile_attribute('LithCode')=='E':  # evaporites
        #if point.get_shapefile_attribute('LithCode')=='T' or point.get_shapefile_attribute('LithCode')=='D' or point.get_shapefile_attribute('LithCode')=='G': # glacial deposits
        #if point.get_shapefile_attribute('LithCode')=='C' or point.get_shapefile_attribute('LithCode')=='E' or point.get_shapefile_attribute('LithCode')=='T' or point.get_shapefile_attribute('LithCode')=='D' or point.get_shapefile_attribute('LithCode')=='G': # the three types
            point_features_onetype.append(point)
            features_collection.append(point)
    print (len(point_features_onetype))
    #output_feature_collection = pygplates.FeatureCollection(point_features_onetype)
    #output_feature_collection.write('PresentDay_LithData_Matthews2016_CEG_%sMa.shp' % np.mean(age_range))

output_feature_collection = pygplates.FeatureCollection(features_collection)
output_feature_collection.write('Output/ProcessingData_1/PresentDay_Coals_Matthews2016_410-0Ma.shp')


../Data/Original_Data\Map06 LDevon v5.csv (419, 393)
3
../Data/Original_Data\Map07 Eifel v5.csv (393, 388)
5
../Data/Original_Data\Map08 Givet v5.csv (388, 383)
11
../Data/Original_Data\Map09 UDevon v5.csv (383, 359)
27
../Data/Original_Data\Map10 TourVise v4.csv (359, 331)
229
../Data/Original_Data\Map11 Serpuk v4.csv (331, 323)
117
../Data/Original_Data\Map12 BashMosc v4.csv (323, 307)
316
../Data/Original_Data\Map13 Steph v4.csv (307, 299)
276
../Data/Original_Data\Map14  AssSak v4.csv (299, 290)
301
../Data/Original_Data\Map15 ArtinLoping v4.csv (290, 252)
340
../Data/Original_Data\Map16 LTrias v4.csv (252, 247)
19
../Data/Original_Data\Map17 MTrias v4.csv (247, 237)
22
../Data/Original_Data\Map18 UTrias v4.csv (237, 201)
234
../Data/Original_Data\Map19 LMJuras v4.csv (201, 164)
489
../Data/Original_Data\Map20 UJuras v4.csv (164, 145)
88
../Data/Original_Data\Map21 preAlbian  v6.csv (145, 113)
252
../Data/Original_Data\Map22 AlbianTuron v5.csv (113, 89.8)
179
../Data/Original_Data\